# Traffic Watch NI Utilities

* Enumerate available cameras
* Grab images from cameras
* Combine images into a single grid image
* Flask service to serve cached images with limited interactivity

In [2]:
%pip install beautifulsoup4


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import requests
from bs4 import BeautifulSoup as bs


base_url = "https://www.trafficwatchni.com"
camera_url = f"{base_url}/twni/cameras/"

response = requests.get(camera_url)
soup = bs(response.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   TrafficWatchNI - Cameras
  </title>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <!-- <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" /> -->
  <meta content="width=device-width, initial-scale=1, maximum-scale=500" name="viewport"/>
  <script src="/twni/javascript/js-cookie/js.cookie.js?v=29092022">
  </script>
  <!-- Google Analytics now using Google tag (gtag.js) -->
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-069FVZSKG2">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];
		  function gtag(){dataLayer.push(arguments);}
		  gtag('js', new Date());
		
		  gtag('config', 'G-069FVZSKG2');
  </script>
  <!-- End Google Analytics -->
  <link href="/twni/css/fontawesome-all.min.css?v=29092022" rel="stylesheet"/>
  <link href="/twni/javascript/bootstrap/bootstrap.min.css?v=27062023" rel="sty

In [14]:
# From the above soup entity, there is a div with the id "cctvSelection" which contains the cameras; each camera element is in a div with the class "cctvOption"; and these cctv options are grouped into divs of class "camera-group" container, each with a header div "camera-group-name" which includes the name of the camera group. 
# Construct a dictionary of the cameras, with the camera group name as the key and a list of the cameras in that group as the value.

camera_groups = {}
for group in soup.find_all('div', class_='camera-group-container'):
    group_name = group.find('div', class_='camera-group-name').text
    cameras = group.find_all('div', class_='cctvOption')
    camera_groups[group_name] = cameras

print(camera_groups)

{'Greater Belfast': [<div class="cctvOption">
<div class="imgrotate" id="imgrotate">
<!-- First Link -->
<a class="pb-1 text-nowrap text-truncate align-middle cctv-no-style" href="/twni/cameras/static?id=44" id="cameraLink" title="View camera : A2 - Tillysburn">A2 - Tillysburn</a>
<!-- Clickable Link -->
<button class="cameraNewTabLink" onclick="var childWindow = window.open('/twni/cameras/static?id=44', 'childWindow', 'width=700,height=600,top=100,left=100,scrollbars=yes,resizable=yes'); childWindow.focus();" title="Button to open the camera in a separate window" type="button">
<img alt="open in new tab" src="/twni/images/pages/cameras_open_in_new_tab.png" title="View camera : A2 - Tillysburn"/>
</button>
<!-- Button -->
<button class="cameraPreviewAdd" onclick='addToPreview(44, "https:\/\/cctv.trafficwatchni.com", "1734545910000", "3260.jpg", "A2 - Tillysburn")' title="Click here to add this camera to the Favourites Panel">
<img alt="Add" src="/twni/images/pages/wired-outline-49-plus

In [25]:
cameras[0]

<div class="cctvOption">
<div class="imgrotate" id="imgrotate">
<!-- First Link -->
<a class="pb-1 text-nowrap text-truncate align-middle cctv-no-style" href="/twni/cameras/static?id=13" id="cameraLink" title="View camera : M1 Tamnamore">M1 Tamnamore</a>
<!-- Clickable Link -->
<button class="cameraNewTabLink" onclick="var childWindow = window.open('/twni/cameras/static?id=13', 'childWindow', 'width=700,height=600,top=100,left=100,scrollbars=yes,resizable=yes'); childWindow.focus();" title="Button to open the camera in a separate window" type="button">
<img alt="open in new tab" src="/twni/images/pages/cameras_open_in_new_tab.png" title="View camera : M1 Tamnamore"/>
</button>
<!-- Button -->
<button class="cameraPreviewAdd" onclick='addToPreview(13, "https:\/\/cctv.trafficwatchni.com", "1734545910000", "17.jpg", "M1 Tamnamore")' title="Click here to add this camera to the Favourites Panel">
<img alt="Add" src="/twni/images/pages/wired-outline-49-plus-circle-static.png"/>
</button>
</d

In [40]:
# Extract the 'onclick' argument from the 'cameraPreviewAdd' button.
# The 'onclick' argument contains the camera id, a base url, a cache timestamp, camera image filename and a camera name.

camera = cameras[0]

def parse_cctvoption(camera):
    onclick = camera.find('button', class_='cameraPreviewAdd')['onclick']
    try:
        _, cctv_url, cache_key, file, camera_name = eval(onclick.replace("addToPreview", ""))
    except ValueError:
        print(f"Error parsing camera: {onclick}")
        raise
    camera_id = file.split("_")[0]
    return {
        "camera_id": camera_id,
        "cctv_url": f"https://cctv.trafficwatchni.com/{file}",
        "cache_key": cache_key,
        "file": file,
        "camera_name": camera_name
    }

parse_cctvoption(camera)


{'camera_id': '55.jpg',
 'cctv_url': 'https://cctv.trafficwatchni.com/55.jpg',
 'cache_key': '1734545910000',
 'file': '55.jpg',
 'camera_name': 'A2 Shore Rd - Trooperslane, Carrick'}

In [41]:
camera_groups = {}
for group in soup.find_all('div', class_='camera-group-container'):
    group_name = group.find('div', class_='camera-group-name').text
    cameras = group.find_all('div', class_='cctvOption')
    cameras = [{'camera_group':group_name, **parse_cctvoption(camera)} for camera in cameras]
    camera_groups[group_name] = cameras

print(camera_groups)

{'Greater Belfast': [{'camera_group': 'Greater Belfast', 'camera_id': '3260.jpg', 'cctv_url': 'https://cctv.trafficwatchni.com/3260.jpg', 'cache_key': '1734545910000', 'file': '3260.jpg', 'camera_name': 'A2 - Tillysburn'}, {'camera_group': 'Greater Belfast', 'camera_id': '3261.jpg', 'cctv_url': 'https://cctv.trafficwatchni.com/3261.jpg', 'cache_key': '1734545910000', 'file': '3261.jpg', 'camera_name': 'A2 - Holywood Esplanade'}, {'camera_group': 'Greater Belfast', 'camera_id': '8.jpg', 'cctv_url': 'https://cctv.trafficwatchni.com/8.jpg', 'cache_key': '1734545910000', 'file': '8.jpg', 'camera_name': 'A12 Clifton Street'}, {'camera_group': 'Greater Belfast', 'camera_id': '4.jpg', 'cctv_url': 'https://cctv.trafficwatchni.com/4.jpg', 'cache_key': '1734545910000', 'file': '4.jpg', 'camera_name': 'A12 Roden Street'}, {'camera_group': 'Greater Belfast', 'camera_id': '106.jpg', 'cctv_url': 'https://cctv.trafficwatchni.com/106.jpg', 'cache_key': '1734545910000', 'file': '106.jpg', 'camera_name'

In [44]:
from IPython.display import Image, display
group = 'Greater Belfast'

for camera in camera_groups[group]:
    print(camera['camera_name'])
    display(Image(url=camera['cctv_url']))


A2 - Tillysburn


A2 - Holywood Esplanade


A12 Clifton Street


A12 Roden Street


M1 - Stockmans Lane


...

M3 - Lagan Bridge East


M3 Lagan Bridge


Albertbridge Road - Woodstock Link


Oxford Street - Lanyon Place


Donegall Square South-Adelaide Street
